# Fit the Model to Training Data

## Setup

In [ ]:
from importlib import reload

import numpy as np
import tensorflow as tf

from hot_dust import preprocess, model

In [ ]:
# "reload" to get changes in preprocess.py without restarting the kernel
reload(preprocess)
reload(model)
from hot_dust.preprocess import prepare_training_data, split_training_data
from hot_dust.model import to_tensorflow, compile, pretraining

### Parameters

In [ ]:
epochs = 500  # Max 500
batch_size = 64
buffer_size = 10 * batch_size

ds = prepare_training_data()
train, validate, test = to_tensorflow(split_training_data(ds))

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size).cache()

In [ ]:
layer = pretraining(dataset=test)
network = compile(normalization=layer)
network.summary()

### Fitting

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
)

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
    callbacks=[early_stopping],
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", fit)

### Training and Validation Losses

In [ ]:
model.plot_loss(fit)